run in cmd :  ollama run qwen3:8b
then ollama serve

In [ ]:
LLM            ='qwen3:8b'
semantic_model = 'https://raw.githubusercontent.com/djouallah/Fabric_Notebooks_Demo/refs/heads/main/SemanticModel/sm/semantic_model.txt' 

In [2]:
import requests
import json
import duckdb
import pathlib


In [3]:
def get_ollama_response(user_message,  ollama_url='http://127.0.0.1:11434/api/chat'):
    try:
        github_response = requests.get(semantic_model)
        github_response.raise_for_status()
        system_prompt = github_response.text.strip()
    except requests.RequestException as e:
        return f"Error fetching system prompt from GitHub: {e}"

    # Prepare payload for Ollama
    payload = {
        'model': LLM,
        'messages': [
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': user_message}
        ],
        'stream': False
    }

    # Send request to Ollama
    try:
        response = requests.post(ollama_url, json=payload)
        response.raise_for_status()
        return json.loads(response.text)['message']['content'].replace("<think>", "").replace("</think>", "")
    except requests.RequestException as e:
        return f"Error with Ollama API request: {e}"

In [4]:
def execute_sql_with_retry(query: str, max_attempts: int = 1) -> str:
    """
    Executes a SQL query and retries with Ollama-corrected query if a syntax error occurs.
    
    Args:
        con: SQL connection object (e.g., DuckDB connection).
        query: The SQL query to execute.
        github_url: Raw URL of the GitHub text file with the system prompt for SQL correction.
        max_attempts: Maximum number of retry attempts (default: 3).
    
    Returns:
        The query result as a string, or an error message if all attempts fail.
    """
    attempt = 1
    current_query = query.strip()

    while attempt <= max_attempts:
        try:
            # Execute the SQL query
            result = con.sql(current_query).show()
            return str(result)  # Return the result as a string
        except Exception as e:  # DuckDB uses generic Exception for syntax errors; adjust for your engine
            if "syntax" not in str(e).lower() and "parser" not in str(e).lower():
                return f"Non-syntax error: {e}"  # Exit if error is not syntax-related

            print(f"Attempt {attempt}/{max_attempts} failed with syntax error: {e}")
            if attempt == max_attempts:
                return f"Max attempts reached. Last error: {e}"

            # Prepare message for Ollama to fix the query
            ollama_message = (
                f"The following SQL query has a syntax error: '{current_query}'.\n"
                f"Error message: {e}\n"
                f"Please provide the corrected SQL query. Return only the corrected query without explanation."
            )

            # Get corrected query from Ollama
            corrected_query = get_ollama_response( ollama_message)
            if corrected_query.startswith("Error"):
                return f"Failed to get corrected query from Ollama: {corrected_query}"

            print(f"Received corrected query: {corrected_query}")
            current_query = corrected_query.strip()
            attempt += 1


In [ ]:
SF = 0.1  # Scale Factor
if SF <1 :
 schema = f"{str(SF).replace('.', '_')}"
else :
 schema = f'DS{SF:02d}'
import os
folder_path = '/some_location'
os.makedirs(folder_path, exist_ok=True)
db_path = folder_path +"/"+ schema +".duckdb"
if not pathlib.Path(db_path).exists():
    con = duckdb.connect(db_path)
    con.sql("SET memory_limit = '14GB' ")
    con.sql(f"CALL dsdgen(sf={SF})")
    con.close()
con = duckdb.connect()
con.sql(f""" attach or replace '{db_path}' as ds(read_only) ; use ds """)

In [7]:
%%time
for x in [
          'return rate per year',
          'customer age group with the worst return rate?',
          'any days with unusual return rate?, use fancy statistics',
          'total sales by region',
          "Identify the top 10 item categories with the highest total return amount from customers born in 'USA' who made returns in 2001.",
        ]:
    sql_query = get_ollama_response(x)
    result = execute_sql_with_retry(sql_query,max_attempts=3)
    print(result)
    print(sql_query)

┌───────┬───────────────┬───────────────┬────────────────────┐
│ year  │  total_sales  │ total_returns │    return_rate     │
│ int32 │ decimal(38,2) │ decimal(38,2) │       double       │
├───────┼───────────────┼───────────────┼────────────────────┤
│  1998 │  102297175.54 │    3174362.68 │ 3.1030794968124686 │
│  1999 │   99370448.87 │    5123614.42 │  5.156074545565248 │
│  2000 │  103184045.02 │    5280670.28 │  5.117719778262673 │
│  2001 │  102172898.92 │    5228059.14 │ 5.1168746265029625 │
│  2002 │  100289052.41 │    5118797.44 │  5.104044077586275 │
│  2003 │     873774.86 │    2222881.70 │ 254.39982331375387 │
└───────┴───────────────┴───────────────┴────────────────────┘

None




-- return rate per year
WITH store_sales_agg AS (
    SELECT
        d.d_year,
        SUM(ss.ss_sales_price * ss.ss_quantity) AS total_sales
    FROM store_sales AS ss
    JOIN date_dim AS d ON ss.ss_sold_date_sk = d.d_date_sk
    GROUP BY d.d_year
), store_returns_agg AS (
    SELECT
        d.